In [31]:
import open3d as o3d
import numpy as np
import copy

In [39]:
# Voxel carving

def xyz_spherical(xyz):
    x = xyz[0]
    y = xyz[1]
    z = xyz[2]
    r = np.sqrt(x * x + y * y + z * z)
    r_x = np.arccos(y / r)
    r_y = np.arctan2(z, x)
    return [r, r_x, r_y]


def get_rotation_matrix(r_x, r_y):
    rot_x = np.asarray([[1, 0, 0], [0, np.cos(r_x), -np.sin(r_x)],
                        [0, np.sin(r_x), np.cos(r_x)]])
    rot_y = np.asarray([[np.cos(r_y), 0, np.sin(r_y)], [0, 1, 0],
                        [-np.sin(r_y), 0, np.cos(r_y)]])
    return rot_y.dot(rot_x)


def get_extrinsic(xyz):
    rvec = xyz_spherical(xyz)
    r = get_rotation_matrix(rvec[1], rvec[2])
    t = np.asarray([0, 0, 2]).transpose()
    trans = np.eye(4)
    trans[:3, :3] = r
    trans[:3, 3] = t
    return trans


def preprocess(model):
    min_bound = model.get_min_bound()
    max_bound = model.get_max_bound()
    center = min_bound + (max_bound - min_bound) / 2.0
    scale = np.linalg.norm(max_bound - min_bound) / 2.0
    vertices = np.asarray(model.vertices)
    vertices -= center
    model.vertices = o3d.utility.Vector3dVector(vertices / scale)
    return model


def voxel_carving(obj,
                  output_filename,
                  camera_path,
                  cubic_size,
                  voxel_resolution,
                  w=300,
                  h=300,
                  use_depth=True,
                  surface_method='pointcloud'):
    obj.compute_vertex_normals()
    camera_sphere = o3d.io.read_triangle_mesh(camera_path)

    # setup dense voxel grid
    voxel_carving = o3d.geometry.VoxelGrid.create_dense(
        width=cubic_size,
        height=cubic_size,
        depth=cubic_size,
        voxel_size=cubic_size / voxel_resolution,
        origin=[-cubic_size / 2.0, -cubic_size / 2.0, -cubic_size / 2.0],
        color=[1.0, 0.7, 0.0])

    # rescale geometry
    camera_sphere = preprocess(camera_sphere)
    obj = preprocess(obj)

    # setup visualizer to render depthmaps
    vis = o3d.visualization.Visualizer()
    vis.create_window(width=w, height=h, visible=False)
    vis.add_geometry(obj)
    vis.get_render_option().mesh_show_back_face = True
    ctr = vis.get_view_control()
    param = ctr.convert_to_pinhole_camera_parameters()

    # carve voxel grid
    pcd_agg = o3d.geometry.PointCloud()
    centers_pts = np.zeros((len(camera_sphere.vertices), 3))
    for cid, xyz in enumerate(camera_sphere.vertices):
        # get new camera pose
        trans = get_extrinsic(xyz)
        param.extrinsic = trans
        c = np.linalg.inv(trans).dot(np.asarray([0, 0, 0, 1]).transpose())
        centers_pts[cid, :] = c[:3]
        ctr.convert_from_pinhole_camera_parameters(param)

        # capture depth image and make a point cloud
        vis.poll_events()
        vis.update_renderer()
        depth = vis.capture_depth_float_buffer(False)
        pcd_agg += o3d.geometry.PointCloud.create_from_depth_image(
            o3d.geometry.Image(depth),
            param.intrinsic,
            param.extrinsic,
            depth_scale=1)

        # depth map carving method
        if use_depth:
            voxel_carving.carve_depth_map(o3d.geometry.Image(depth), param)
        else:
            voxel_carving.carve_silhouette(o3d.geometry.Image(depth), param)
        print("Carve view %03d/%03d" % (cid + 1, len(camera_sphere.vertices)))
    vis.destroy_window()

    # add voxel grid survace
    print('Surface voxel grid from %s' % surface_method)
    if surface_method == 'pointcloud':
        voxel_surface = o3d.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
            pcd_agg,
            voxel_size=cubic_size / voxel_resolution,
            min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
            max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2))
    elif surface_method == 'mesh':
        voxel_surface = o3d.geometry.VoxelGrid.create_from_triangle_mesh_within_bounds(
            obj,
            voxel_size=cubic_size / voxel_resolution,
            min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
            max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2))
    else:
        raise Exception('invalid surface method')
    voxel_carving_surface = voxel_surface + voxel_carving

    return voxel_carving_surface, voxel_carving, voxel_surface

In [40]:
import os

base_obj = o3d.io.read_triangle_mesh('../data/suzanne.stl')

output_filename = os.path.abspath("../data/voxelized.ply")
camera_path = os.path.abspath("../data/sphere.ply")
visualization = True
cubic_size = 1
voxel_resolution = 2**4

voxel_grid, voxel_carving, voxel_surface = voxel_carving(
    base_obj, output_filename, camera_path, cubic_size, voxel_resolution)

Carve view 001/642
Carve view 002/642
Carve view 003/642
Carve view 004/642
Carve view 005/642
Carve view 006/642
Carve view 007/642
Carve view 008/642
Carve view 009/642
Carve view 010/642
Carve view 011/642
Carve view 012/642
Carve view 013/642
Carve view 014/642
Carve view 015/642
Carve view 016/642
Carve view 017/642
Carve view 018/642
Carve view 019/642
Carve view 020/642
Carve view 021/642
Carve view 022/642
Carve view 023/642
Carve view 024/642
Carve view 025/642
Carve view 026/642
Carve view 027/642
Carve view 028/642
Carve view 029/642
Carve view 030/642
Carve view 031/642
Carve view 032/642
Carve view 033/642
Carve view 034/642
Carve view 035/642
Carve view 036/642
Carve view 037/642
Carve view 038/642
Carve view 039/642
Carve view 040/642
Carve view 041/642
Carve view 042/642
Carve view 043/642
Carve view 044/642
Carve view 045/642
Carve view 046/642
Carve view 047/642
Carve view 048/642
Carve view 049/642
Carve view 050/642
Carve view 051/642
Carve view 052/642
Carve view 0

In [41]:
print("surface voxels")
print(voxel_surface)
o3d.visualization.draw_geometries([voxel_surface])

# print("carved voxels")
# print(voxel_carving)
# o3d.visualization.draw_geometries([voxel_carving])

# print("combined voxels (carved + surface)")
# print(voxel_grid)
# o3d.visualization.draw_geometries([voxel_grid])

surface voxels
VoxelGrid with 1225 voxels.


In [42]:
# you can acces the voxels by voxel_grid.get_voxels()
voxels = voxel_grid.get_voxels()
voxels = np.array(voxels)
print(voxels.shape)

(2028,)


In [46]:
vox_mesh=o3d.geometry.TriangleMesh()

# meta_obj = o3d.io.read_triangle_mesh('../data/meta.stl')
# meta_obj =  preprocess(meta_obj)

for v in voxels:
    # cube=o3d.geometry.TriangleMesh.create_box(width=1, height=1,
    # depth=1)
    # meta_obj = cube
    meta_obj = o3d.io.read_triangle_mesh('../data/meta.stl')
    # meta_obj =  preprocess(meta_obj)
    # meta_obj.paint_uniform_color(v.color)
    meta_obj.translate(v.grid_index, relative=False)
    vox_mesh+=meta_obj

# Final
# vox_mesh.translate([0.5,0.5,0.5], relative=True)
# vox_mesh.scale(voxel_size, [0,0,0])
# vox_mesh.translate(voxel_grid.origin, relative=True)
# vox_mesh.merge_close_vertices(0.0000001)
vox_mesh = o3d.geometry.TriangleMesh.compute_triangle_normals(vox_mesh)



In [47]:
o3d.visualization.draw_geometries([vox_mesh])
# o3d.io.write_triangle_mesh("../data/"+"combined.stl", vox_mesh)

In [11]:
import stl
from stl import mesh
from matplotlib import pyplot
from mpl_toolkits import mplot3d

In [15]:
# Build mesh using numpy-stl

meta_path =  '../data/meta.stl'
resolution = 1

def show_stl(mesh: mesh.Mesh):
    figure = pyplot.figure()
    axes = mplot3d.Axes3D(figure, auto_add_to_figure=False)
    figure.add_axes(axes)
    axes.add_collection3d(mplot3d.art3d.Poly3DCollection(mesh.vectors))
    # Auto-scale plot to mesh scale
    scale = combined.points.flatten()
    axes.auto_scale_xyz(scale, scale, scale)   
    pyplot.show() 
    
meta = mesh.Mesh.from_file(meta_path)
# We need to scale the metastrucure to the pitch
ratio = resolution/(meta.x.max() - meta.x.min())
meta.points *= ratio

copies = []
for v in voxels:
    p = v.grid_index
    copy = mesh.Mesh(meta.data.copy())
    copy.translate(p)
    copies.append(copy)

combined = mesh.Mesh( np.concatenate( [copy.data for copy in copies]) )

# show_stl(combined)
combined.save('../data/combined.stl', mode=stl.Mode.ASCII)  # save as ASCII
